In [1]:
import warnings
warnings.filterwarnings('ignore')
import picaso.justdoit as jdi
import astropy.units as u
import numpy as np
import tracemalloc
import time
import matplotlib.pyplot as plt
from scipy import special
import os.path

In [2]:
def guillot_pt(Teq, T_int,grav, logg1, logKir, alpha,nlevel, p_bottom, p_top):
    kv1, kv2 =10.**(logg1+logKir),10.**(logg1+logKir)
    kth=10.**logKir

    Teff = T_int
    f = 1.0  # solar re-radiation factor
    A = 0.0  # planetary albedo
    g0 = grav #cm/s2 to m/s2

    # Compute equilibrium temperature and set up gamma's
    T0 = Teq
    gamma1 = kv1/kth #Eqn. 25
    gamma2 = kv2/kth

    # Initialize arrays
    logtau =np.arange(-10,20,.1)
    tau =10**logtau

    #computing temperature
    T4ir = 0.75*(Teff**(4.))*(tau+(2.0/3.0))
    f1 = 2.0/3.0 + 2.0/(3.0*gamma1)*(1.+(gamma1*tau/2.0-1.0)*np.exp(-gamma1*tau))+2.0*gamma1/3.0*(1.0-tau**2.0/2.0)*special.expn(2.0,gamma1*tau)
    f2 = 2.0/3.0 + 2.0/(3.0*gamma2)*(1.+(gamma2*tau/2.0-1.0)*np.exp(-gamma2*tau))+2.0*gamma2/3.0*(1.0-tau**2.0/2.0)*special.expn(2.0,gamma2*tau)

    T4v1=f*0.75*T0**4.0*(1.0-alpha)*f1
    T4v2=f*0.75*T0**4.0*alpha*f2
    T=(T4ir+T4v1+T4v2)**(0.25)
    P=tau*g0/(kth*0.1)/1.E5

    logP = np.linspace(p_top,p_bottom,nlevel)
    newP = 10.0**logP
    T = np.interp(logP,np.log10(P),T)
    
    return T   

In [9]:
def run_planet(T_int,grav,a,T_star,logg,metal,rstar,mh,CtoO,rfacv):
    mh = mh
    CtoO = CtoO

    filename_db="/Users/sagnickmukherjee/Documents/GitHub/picaso-dev/reference/opacities/ck_cx_cont_opacities.db"
    ck_db='/Users/sagnickmukherjee/Documents/software/picaso-dev/reference/opacities/ck_db/m'+mh+'_co'+CtoO+'.data.196'

    
    path = "/Users/sagnickmukherjee/Documents/research/geers/wasp39b"
    file_path= path+"/tpfile/tp_eq_planet_"+str(int(T_int))+"_grav_4.5_mh_"+mh+"_CO_"+CtoO+"_sm_0.0486_v_0.5_.dat"
    pressure,temp_guess = np.loadtxt(file_path,usecols=[0,1],unpack=True)
    file_temp = open(file_path)
    header = file_temp.readline()
    print(header)
    if int(header[18]) == 0:
        nstr1 = int(header[11]+header[12])+1
    else:
        nstr1 = 83
    print(nstr1)
    opacity_ck = jdi.opannection(ck=True, ck_db=ck_db,filename_db=filename_db)#, deq = True, first_deq = True)


    # planet prop
    semi_major = a #0.0355 #"NA" # AU
    T_eff = T_int
    grav = grav
    r_planet = 1 # needed for vulcan photochem runs

    ### star prop
    T_star = T_star
    logg = logg
    metal = metal
    r_star = rstar




    nlevel = 91
    #pressure=np.logspace(np.log10(1e-6),np.log10(200.0),nlevel)
    #temp_guess = np.zeros(shape=(nlevel)) 
    kz = np.zeros(shape =(nlevel))
    #pressure_dummy,temp_guess,t1,t2,t3,t4 = np.loadtxt("/data/users/samukher/Disequilibrium-picaso/structures_m+0.0/t"+str(round(600/100.0)*100)+"g"+str(31)+"nc_m0.0.dat",usecols=[1,2,3,4,5,6],unpack=True, skiprows = 1)
    #Teq = T_star*np.sqrt(rstar/(2*semi_major*216))
    #temp_guess = guillot_pt(Teq=Teq, T_int=T_int,grav=grav, logg1=-1, logKir=-1.5, alpha=0.5,nlevel=91, p_bottom = np.log10(np.max(pressure)), p_top =np.log10(np.min(pressure)))

    #pdummy,temp_guess = np.loadtxt(path+"tpfile/tp_eq_planet_200_grav_4.5_mh_+0.0_CO_0.5_sm_0.0486_v_0.5_.dat",usecols=[0,1],unpack=True)
    
    nofczns = 1
    #nstr1 = 83
    nstr = np.array([0,nstr1,89,0,0,0])
    rfacv = rfacv
    rfaci =1.0
    cloudy = False
    fsed = 2




    ###### run start

    cl_run = jdi.inputs(calculation="planet", climate = True)

    cl_run.star(opacity_ck, temp =T_star,metal =metal, logg =logg, radius = r_star, radius_unit=u.R_sun,semi_major= semi_major , semi_major_unit = u.AU)#opacity db, pysynphot database, temp, metallicity, logg

    cl_run.gravity(gravity=grav, gravity_unit=u.Unit('m/(s**2)'))
    cl_run.T_eff(Teff=T_eff)





    cl_run.inputs_climate(temp_guess= temp_guess, pressure= pressure, nstr = nstr, nofczns = nofczns , rfacv = rfacv, rfaci = rfaci, cloudy = cloudy, mh = mh, CtoO = CtoO,species = ['MgSiO3','Mg2SiO4','Fe'], fsed = fsed, T_star = T_star, logg = logg, metal = metal, r_star = r_star, semi_major = semi_major)

    #tracemalloc.start()
    t0 = time.time()
    pressure , temp, dtdp, nstr_new, flux_plus_final, df, all_profiles,opd_cld,g0_cld,w0_cld = cl_run.run_climate_model(opacity_ck,save_all_profiles=True,diseq_chem = False, self_consistent_kzz =False, kz = kz )
    t1 = time.time()
    print(t1-t0)
   
    quench_levels=np.array([0,0,0,0])
    path = "/Users/sagnickmukherjee/Documents/research/geers/wasp39b/"

    filename= path+"tpfile_new/tp_eq_planet_"+str(T_eff)+"_grav_"+str(grav)+"_mh_"+mh+"_CO_"+CtoO+"_sm_"+str(semi_major)+"_v_"+str(rfacv)+"_.dat"
    np.savetxt(filename, np.transpose([pressure,temp]), header= str("nstr "+str(nstr_new)+" T_eff "+str(T_eff)+" grav "+ str(grav)+" kz "+str(kz[0])+" T_star "+str(T_star)+" semi-major[AU] "+str(semi_major)))        

    csv = path+"profile_new/profile_eq_planet_"+str(T_eff)+"_grav_"+str(grav)+"_mh_"+mh+"_CO_"+CtoO+"_sm_"+str(semi_major)+"_v_"+str(rfacv)+"_.dat"
    df.to_csv(csv,sep='\t')

    hist = path+"hist_new/hist_eq_planet_"+str(T_eff)+"_grav_"+str(grav)+"_mh_"+mh+"_CO_"+CtoO+"_sm_"+str(semi_major)+"_v_"+str(rfacv)+"_.dat"
    np.savetxt(hist, all_profiles)
    
    return t1-t0

In [10]:

#T_int = np.array([100,200,300])
T_int=np.array([100])#,200,300])
grav = np.array([4.26])
T_star,logg,metal,rstar = 5326.6,4.38933,-0.03,0.932
mh = np.array(['-1.0','-0.5','+0.0','+0.5','+1.0','+1.5','+1.7','+2.0'])
CtoO =np.array(['0.5','1.0','1.5','2.0'])


a,e = 0.0486, 0
cosv = np.array([0])

semi_major_ar = np.array([a])
rfacv = np.array([0.5,0.4])

for i in range(len(T_int)):
    for j in range(len(grav)):
        for k in range(len(mh)):
            for l in range(len(CtoO)):
                for m in range(len(semi_major_ar)):
                    for o in range(len(rfacv)):
                        t= run_planet(T_int[i],grav[j],semi_major_ar[m],T_star,logg,metal,rstar,mh[k],CtoO[l],rfacv[o])
    


# nstr [ 0 87 89  0  0  0] T_eff 100 grav 4.5 kz 0.0 T_star 5326.6 semi-major[AU] 0.0486

88
Iteration number  0 , min , max temp  677.418416514663 2303.2394198482716 , flux balance  0.1373564139586042
Iteration number  1 , min , max temp  677.4118126161381 2301.2234476706117 , flux balance  3.678337774452047e-05
Converged Solution in iterations  1
Big iteration is  677.4118126161381 0
Iteration number  0 , min , max temp  677.4114972173119 2336.2032728458166 , flux balance  0.00336170764116378
Iteration number  1 , min , max temp  677.4114994617345 2334.940758819926 , flux balance  5.377193383079755e-07
Converged Solution in iterations  1
Profile converged
Iteration number  0 , min , max temp  677.4074646496152 2359.5057384800803 , flux balance  0.0006916603916592217
Iteration number  1 , min , max temp  677.4074641047478 2358.901136542896 , flux balance  1.1212133448856384e-07
Converged Solution in iterations  1
Big iteration is  677.4074641047478 0
Iteration number  0 , min , max te

Profile converged
YAY ! ENDING WITH CONVERGENCE
130.4911470413208
# nstr [ 0 87 89  0  0  0] T_eff 100 grav 4.5 kz 0.0 T_star 5326.6 semi-major[AU] 0.0486

88
Iteration number  0 , min , max temp  648.5821720947807 2324.2072275402775 , flux balance  0.14086640051577223
Iteration number  1 , min , max temp  648.5764947592842 2321.1099825888396 , flux balance  4.0034623587454e-05
Converged Solution in iterations  1
Big iteration is  648.5764947592842 0
Iteration number  0 , min , max temp  648.1420650946934 2366.9337141170736 , flux balance  0.010459594079320041
Iteration number  1 , min , max temp  648.14159489819 2364.8614423573845 , flux balance  1.8296834242541343e-06
Converged Solution in iterations  1
Profile converged
Iteration number  0 , min , max temp  647.9847469520582 2397.324645117625 , flux balance  0.0024571752630022413
Iteration number  1 , min , max temp  647.9846728721669 2396.3182634467444 , flux balance  3.1886913011318887e-07
Converged Solution in iterations  1
Big i

Iteration number  0 , min , max temp  618.0037768994937 2278.807809867636 , flux balance  -8.86653658057733e-07
Converged Solution in iterations  0
Profile converged
YAY ! ENDING WITH CONVERGENCE
128.02450585365295
# nstr [ 0 86 89  0  0  0] T_eff 100 grav 4.5 kz 0.0 T_star 5326.6 semi-major[AU] 0.0486

87
Iteration number  0 , min , max temp  703.0120729337023 2474.7583872749547 , flux balance  0.12894917849759455
Iteration number  1 , min , max temp  703.0102792571477 2474.252891221891 , flux balance  3.0319798311754198e-05
Converged Solution in iterations  1
Big iteration is  703.0102792571477 0
Iteration number  0 , min , max temp  703.0298822827132 2487.480874754174 , flux balance  0.004511339963455094
Iteration number  1 , min , max temp  703.0298879874614 2487.314132337811 , flux balance  5.931370320100052e-07
Converged Solution in iterations  1
Profile converged
Iteration number  0 , min , max temp  703.0283444329697 2497.1908003020185 , flux balance  0.0011808935773710073
Conv

Iteration number  0 , min , max temp  645.8617256431655 2472.4916889062806 , flux balance  0.0005597825626517486
Converged Solution in iterations  0
Big iteration is  645.8617256431655 0
Iteration number  0 , min , max temp  645.8634122306622 2472.4901357342037 , flux balance  0.00034373485945501025
Converged Solution in iterations  0
Profile converged
YAY ! ENDING WITH CONVERGENCE
117.44079518318176
# nstr [ 0 86 89  0  0  0] T_eff 100 grav 4.5 kz 0.0 T_star 5326.6 semi-major[AU] 0.0486

87
Iteration number  0 , min , max temp  669.6812451687443 2502.7646560239255 , flux balance  0.1161226848369744
Iteration number  1 , min , max temp  669.6791114375467 2502.0112906015083 , flux balance  3.147153844110985e-05
Converged Solution in iterations  1
Big iteration is  669.6791114375467 0
Iteration number  0 , min , max temp  669.4368410196798 2530.370016509084 , flux balance  0.014665360340298318
Iteration number  1 , min , max temp  669.4366761388292 2529.6375479210474 , flux balance  2.05

Iteration number  0 , min , max temp  629.7632025356919 2492.8327125180413 , flux balance  0.001843964259751573
Converged Solution in iterations  0
Big iteration is  629.7632025356919 0
Iteration number  0 , min , max temp  629.7615831789913 2492.8321139653362 , flux balance  0.00010070892684934417
Converged Solution in iterations  0
Profile converged
final [ 0 86 89  0  0  0]
Iteration number  0 , min , max temp  629.7625690285084 2492.8315269552677 , flux balance  0.00013911698855314248
Converged Solution in iterations  0
Big iteration is  629.7625690285084 0
Iteration number  0 , min , max temp  629.7621601039652 2492.831984467104 , flux balance  -5.298126907208481e-05
Converged Solution in iterations  0
Profile converged
YAY ! ENDING WITH CONVERGENCE
129.88420414924622
# nstr [ 0 84 89  0  0  0] T_eff 100 grav 4.5 kz 0.0 T_star 5326.6 semi-major[AU] 0.0486

85
Iteration number  0 , min , max temp  751.4861251303558 2679.3526745828785 , flux balance  0.15951945924886354
Iteration nu

...

90.45931792259216
# nstr [ 0 81 89  0  0  0] T_eff 100 grav 4.5 kz 0.0 T_star 5326.6 semi-major[AU] 0.0486

82
Iteration number  0 , min , max temp  793.694746681 3429.0973352348788 , flux balance  747.8813770563212
Iteration number  1 , min , max temp  782.179390498908 3434.547852721584 , flux balance  16.864882759028976
Iteration number  2 , min , max temp  781.9216916631703 3434.5340610579615 , flux balance  0.012092389883062982
Converged Solution in iterations  2
Big iteration is  781.9216916631703 0
Iteration number  0 , min , max temp  780.6721243015968 3422.3611842976775 , flux balance  15.169887666501483
Iteration number  1 , min , max temp  780.6676928464469 3422.269191986016 , flux balance  0.018116019561968046
Converged Solution in iterations  1
Big iteration is  780.6676928464469 1
Iteration number  0 , min , max temp  780.5352459712875 3407.2237246000914 , flux balance  0.7854660936912415
Iteration number  1 , min , max temp  780.5350766279513 3407.076718286575 , flux bala

Iteration number  0 , min , max temp  714.9402973208491 3262.0555532976014 , flux balance  0.3932773606176424
Converged Solution in iterations  0
Profile converged
Iteration number  0 , min , max temp  714.4953377818416 3262.1392011177177 , flux balance  0.05420777878746238
Converged Solution in iterations  0
Big iteration is  714.4953377818416 0
Iteration number  0 , min , max temp  714.4664750318625 3262.163088824634 , flux balance  0.004117930192224177
Converged Solution in iterations  0
Profile converged
Iteration number  0 , min , max temp  714.4636748379545 3231.99687103765 , flux balance  0.00013433805758985702
Converged Solution in iterations  0
Big iteration is  714.4636748379545 0
Iteration number  0 , min , max temp  714.4608556564929 3231.996339663325 , flux balance  -9.59577212931995e-06
Converged Solution in iterations  0
Profile converged
Iteration number  0 , min , max temp  714.4598215010782 3236.328563623329 , flux balance  1.2880452595969274e-06
Converged Solution in

Iteration number  0 , min , max temp  787.0310772092782 3526.081555742685 , flux balance  0.02019865376062252
Converged Solution in iterations  0
Profile converged
Iteration number  0 , min , max temp  787.0260883500401 3515.4896880200236 , flux balance  0.0018252424702161919
Converged Solution in iterations  0
Big iteration is  787.0260883500401 0
Iteration number  0 , min , max temp  787.0243856031102 3515.025877092563 , flux balance  0.00023785875892840889
Converged Solution in iterations  0
Profile converged
final [ 0 80 89  0  0  0]
Iteration number  0 , min , max temp  787.0238224260892 3514.8825792923094 , flux balance  5.0147043608695654e-05
Converged Solution in iterations  0
Big iteration is  787.0238224260892 0
Iteration number  0 , min , max temp  787.0236387125497 3514.8439034796143 , flux balance  1.3551264299473274e-05
Converged Solution in iterations  0
Profile converged
YAY ! ENDING WITH CONVERGENCE
111.34214806556702
# nstr [ 0 81 89  0  0  0] T_eff 100 grav 4.5 kz 0.

Profile converged
final [ 0 82 89  0  0  0]
Iteration number  0 , min , max temp  721.9132249440395 3072.4572142971065 , flux balance  1.9715892064076306e-07
Converged Solution in iterations  0
Big iteration is  721.9132249440395 0
Iteration number  0 , min , max temp  721.9132491694533 3072.4572133203874 , flux balance  1.715871853057215e-07
Converged Solution in iterations  0
Profile converged
YAY ! ENDING WITH CONVERGENCE
115.30062985420227
# nstr [ 0 82 89  0  0  0] T_eff 100 grav 4.5 kz 0.0 T_star 5326.6 semi-major[AU] 0.0486

83
Iteration number  0 , min , max temp  733.6397712321567 3192.4332620711707 , flux balance  0.06055531510476189
Iteration number  1 , min , max temp  733.6297935235303 3191.706810833443 , flux balance  1.2769215546973048e-05
Converged Solution in iterations  1
Big iteration is  733.6297935235303 0
Iteration number  0 , min , max temp  732.6736002647491 3194.828318457598 , flux balance  0.005424580608397696
Converged Solution in iterations  0
Profile conver

KeyboardInterrupt: 

In [ ]:
#T_int = np.array([100,200,300])
T_int=np.array([200])#,200,300])
grav = np.array([4.26])
T_star,logg,metal,rstar = 5326.6,4.38933,-0.03,0.932
mh = np.array(['-1.0','-0.5','+0.0','+0.5','+1.0','+1.5','+1.7','+2.0'])
CtoO =np.array(['0.5','1.0','1.5','2.0'])


a,e = 0.0486, 0
cosv = np.array([0])

semi_major_ar = np.array([a])
rfacv = np.array([0.5,0.4])

for i in range(len(T_int)):
    for j in range(len(grav)):
        for k in range(len(mh)):
            for l in range(len(CtoO)):
                for m in range(len(semi_major_ar)):
                    for o in range(len(rfacv)):
                        t= run_planet(T_int[i],grav[j],semi_major_ar[m],T_star,logg,metal,rstar,mh[k],CtoO[l],rfacv[o])
    


In [ ]:
#T_int = np.array([100,200,300])
T_int=np.array([300])#,200,300])
grav = np.array([4.26])
T_star,logg,metal,rstar = 5326.6,4.38933,-0.03,0.932
mh = np.array(['-1.0','-0.5','+0.0','+0.5','+1.0','+1.5','+1.7','+2.0'])
CtoO =np.array(['0.5','1.0','1.5','2.0'])


a,e = 0.0486, 0
cosv = np.array([0])

semi_major_ar = np.array([a])
rfacv = np.array([0.5,0.4])

for i in range(len(T_int)):
    for j in range(len(grav)):
        for k in range(len(mh)):
            for l in range(len(CtoO)):
                for m in range(len(semi_major_ar)):
                    for o in range(len(rfacv)):
                        t= run_planet(T_int[i],grav[j],semi_major_ar[m],T_star,logg,metal,rstar,mh[k],CtoO[l],rfacv[o])
    


In [ ]:
'''
#T_int = np.array([100,200,300])
T_int=np.array([300])
grav = np.array([8.25])
T_star,logg,metal,rstar = 5560.000,4.42,0.14,1.098
#mh = np.array(['-1.0','-0.5','+0.0','+0.5','+1.0','+1.7','+1.5','+2.0'])
mh = np.array(['+1.0'])

CtoO =np.array(['1.0'])
a,e = 0.0453, 0

cosv = np.array([0])
#semi_major_ar = a*(1-e**2)/(1+e*cosv)
semi_major_ar = np.array([a])
rfacv = np.array([0.5])

for i in range(len(T_int)):
    for j in range(len(grav)):
        for k in range(len(mh)):
            for l in range(len(CtoO)):
                for m in range(len(semi_major_ar)):
                    for o in range(len(rfacv)):
                        t= run_planet(T_int[i],grav[j],semi_major_ar[m],T_star,logg,metal,rstar,mh[k],CtoO[l],rfacv[o])
    '''


In [ ]:
'''
import pandas as pd
path = "/Users/sagnickmukherjee/Documents/research/geers/wasp96b/profile/"
df1 = pd.read_csv(path+"newRst_profile_eq_planet_200_grav_8.25_mh_+1.5_CO_1.0_sm_0.0453_v_0.5_.dat",delimiter="\t")

df2 = pd.read_csv(path+"profile_eq_planet_200_grav_8.25_mh_+1.5_CO_1.0_sm_0.0453_v_0.5_.dat",delimiter="\t")

plt.figure(figsize=(10,10))
plt.ylim(1e3,1e-6)
#plt.xlim(800,1500)
plt.xlabel("Temperature",fontsize=20)
plt.ylabel("Pressure",fontsize=20)
plt.semilogy(df1['temperature'],df1['pressure'],label="New")
plt.semilogy(df2['temperature'],df2['pressure'],label="Old")
plt.legend(fontsize=20)
plt.tick_params(axis='both',which='major',length =20, width=3,direction='in',labelsize=20)
plt.tick_params(axis='both',which='minor',length =10, width=2,direction='in',labelsize=20)
'''


In [ ]:
'''
plt.figure(figsize=(10,10))
plt.ylim(1e3,1e-6)
plt.xlabel("Temperature",fontsize=20)
plt.ylabel("Pressure",fontsize=20)
plt.semilogy(df1['temperature']-df2['temperature'],df1['pressure'],label="New")
#plt.semilogy(df2['temperature'],df2['pressure'],label="Old")
plt.legend(fontsize=20)
plt.tick_params(axis='both',which='major',length =20, width=3,direction='in',labelsize=20)
plt.tick_params(axis='both',which='minor',length =10, width=2,direction='in',labelsize=20)
'''

In [ ]:
'''
path = "/Users/sagnickmukherjee/Documents/research/geers/wasp96b/tpfile/"
p,t = np.loadtxt(path+"tp_eq_planet_200_grav_8.25_mh_+1.5_CO_1.0_sm_0.0453_v_0.5_.dat",usecols=[0,1],unpack=True)
f = open(path+"tp_eq_planet_200_grav_8.25_mh_+1.5_CO_1.0_sm_0.0453_v_0.5_.dat")
header = f.readline()
print(header)
#df2 = pd.read_csv(path+"profile_eq_planet_200_grav_8.25_mh_+1.5_CO_1.0_sm_0.0453_v_0.5_.dat",delimiter="\t")
'''

In [ ]:
#print(header[9],header[11]+header[12],header[14]+header[15],header[18])